# DB2-Salesforce connector: Tool usage clusters mapping to users

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
object_id = 'ContactToolClusterAssociation__c'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username": "wang159-4j1v@force.com",
    "password": "napoleon0eZ3PQpQqE3C3z4wWqEqKGhQ8",
} #6Cel800D5w000001jiQu8885w0000003U7mRvSj2dp1TvmgHngvv195i9eJLpzFNuMZBHkjibu07pj7Mk18p7MQxBEO318UtbzlBv7BpFQU

In [2]:
import sys
#sys.path.append(lib_dir)
    
import pandas as pd
import time
import datetime

from DB2SalesforceAPI import DB2SalesforceAPI

## Obtain tool information from DB2 

In [3]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/wang159_myrmekes' \
                                               %(sql_login_params['username'], sql_login_params['password']))

class_info_df = pd.read_sql_query('select * from cluster_class_info', engine)
students_info_df = pd.read_sql_query('select * from cluster_students_info', engine)



In [4]:
students_info_df.head(3)

,index,user,class_id,parti_rate
0,0,salas,0,1.0
1,1,trohrbach,0,1.0
2,2,alqadia,0,1.0


## Obtain Salesforce IDs

In [5]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [6]:
# query the Salesforce IDs for contacts and citations. when updating junction objects, these IDs must be used

# get Salesforce ID for contacts
sf_username_df = db_s.query_data('SELECT Id, nanoHUB_username__c FROM Contact where nanoHUB_username__c != NULL')

[Success] Bulk job creation successful. Job ID = 7505w00000Mt5KwAAJ
{"id":"7505w00000Mt5KwAAJ","operation":"query","object":"Contact","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T01:54:51.000+0000","systemModstamp":"2020-06-09T01:54:52.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":0,"retries":0,"totalProcessingTime":0}
{"id":"7505w00000Mt5KwAAJ","operation":"query","object":"Contact","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T01:54:51.000+0000","systemModstamp":"2020-06-09T01:55:01.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":218066,"retries":0,"totalProcessingTime":8618}
[Success] Bulk job completed successfully.


In [7]:
# get Salesforce ID for citations
sf_clusterID_df = db_s.query_data('SELECT Id, ID__c FROM tool_usage_cluster__c')

sf_clusterID_df['ID__c'] = sf_clusterID_df['ID__c'].astype('int')

[Success] Bulk job creation successful. Job ID = 7505w00000Mt5EJAAZ
{"id":"7505w00000Mt5EJAAZ","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T01:55:08.000+0000","systemModstamp":"2020-06-09T01:55:08.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000Mt5EJAAZ","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000ArpYvAAJ","createdDate":"2020-06-09T01:55:08.000+0000","systemModstamp":"2020-06-09T01:55:08.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000Mt5EJAAZ","operation":"query","object":"tool_usage_cluster__c","createdById":"0055w00000ArpYvAAJ","createdDate":"

## Match data with Salesforce format

In [8]:
# Display
display(students_info_df.head(2))
display(sf_username_df.head(2))
display(sf_clusterID_df.head(2))

,index,user,class_id,parti_rate
0,0,salas,0,1.0
1,1,trohrbach,0,1.0


,Id,nanoHUB_username__c
0,0035w000031Vsp1AAC,hubrepo
1,0035w000031Vsp2AAC,support


,ID__c,Id
0,0,a0w5w000009Q7CqAAK
1,1,a0w5w000009Q7CrAAK


In [9]:
# merge dataframes
cc_tolink_df = pd.merge(students_info_df, sf_username_df, how='inner', left_on='user', right_on='nanoHUB_username__c')\
                           .rename(columns={'Id':'SF_ID_contact'})

cc_tolink_df = pd.merge(cc_tolink_df, sf_clusterID_df, how='inner', left_on='class_id', right_on='ID__c')\
                           .rename(columns={'Id':'SF_ID_cluster'})

# display
cc_tolink_df.head(2)

,index,user,class_id,parti_rate,SF_ID_contact,nanoHUB_username__c,ID__c,SF_ID_cluster
0,0,salas,0,1.0,0035w000034Jm4mAAC,salas,0,a0w5w000009Q7CqAAK
1,1,trohrbach,0,1.0,0035w000034Jm55AAC,trohrbach,0,a0w5w000009Q7CqAAK


In [10]:
# create a new column for object name
cc_tolink_df['Name'] = cc_tolink_df.apply(lambda x: '%s_%d'%(x.nanoHUB_username__c, x.ID__c), axis=1)

In [11]:
df_sf = pd.DataFrame()

# Make sure NaN and NaT values are taken care of here
df_sf['Name']         = cc_tolink_df['Name']
df_sf['Contact__c'] = cc_tolink_df['SF_ID_contact']
df_sf['Tool_Usage_Cluster__c'] = cc_tolink_df['SF_ID_cluster']

sf_original_fields = df_sf.columns

# display
df_sf.head(2).T

,0,1
Name,salas_0,trohrbach_0
Contact__c,0035w000034Jm4mAAC,0035w000034Jm55AAC
Tool_Usage_Cluster__c,a0w5w000009Q7CqAAK,a0w5w000009Q7CqAAK


## To Salesforce Sales Cloud CRM

In [12]:
# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)

# specify Salesforce object ID and external ID
db_s.object_id = object_id
db_s.external_id = external_id

Obtained Salesforce access token ...... True


In [13]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000Mt5MCAAZ
hello
[Success] CSV upload successful. Job ID = 7505w00000Mt5MCAAZ
[Success] Closing job successful. Job ID = 7505w00000Mt5MCAAZ


In [14]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000ArpYvAAJ',
 'createdDate': '2020-06-09T01:55:45.000+0000',
 'externalIdFieldName': 'Name',
 'id': '7505w00000Mt5MCAAZ',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 0,
 'object': 'ContactToolClusterAssociation__c',
 'operation': 'upsert',
 'retries': 0,
 'state': 'UploadComplete',
 'systemModstamp': '2020-06-09T01:55:53.000+0000',
 'totalProcessingTime': 0}
